# Cuaderno para las pruebas del MRN PredicticRale.

"IDENTIFICACIÓN DE ANOMALÍAS COVID-19 EN RADIOGRAFÍAS DE TÓRAX MEDIANTE REDES NEURONALES Y EL SISTEMA RALE”

# Marco General de Método Propuesto:
la Siguiente Figura Muestra el método propuesto, que siguió la construcción del modelo de Red ConvRalec, la misma procesa las imágenes y las etiqueta antes de ingresar al modelo de entrenamiento y validación, y del mismo modo para el proceso de pruebas. 

![Marco General de Metodo Propuesto](http://drive.google.com/uc?export=view&id=1Lqeo0jAMkF-Z3RTlh2-2bkwXoIaSLK_7)


# Librerias: 
pandas, os, matplotlib, pydicom, numpy, keras, math, time, tqdm, torch, glob, opencv2, pathlib, keras y tensorflow.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pydicom as pydi
import numpy as np

import math
import time
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.patches as patches

import glob
import random
import sys
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.autograd import Variable
import cv2
import pydicom as pydi
import pathlib

from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

import tensorflow as tf
from sklearn.metrics import accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

print('Segmento Terminado')

# Clases y funciones UTILS YOLO
Funciones de la Red convolucional YOLO para la categorización de cuadrantes de Radiografías de Tórax, PosteroAnterior, preparación de imagen y Transfer, una vez localizado los cuadrantes.

![Clasificacion de Cuadrantes en Caja Torasica](http://drive.google.com/uc?export=view&id=14c9Hva4zIHM144fE_95f7Ppn5kAcXMZ4)



In [ ]:
def to_cpu(tensor):
    return tensor.detach().cpu()


def load_classes(path):
    fp = open(path, "r")
    names = fp.read().split("\n")[:-1]
    return names


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


def rescale_boxes(boxes, current_dim, original_shape):
    orig_h, orig_w = original_shape
    pad_x = max(orig_h - orig_w, 0) * (current_dim / max(original_shape))
    pad_y = max(orig_w - orig_h, 0) * (current_dim / max(original_shape))
    unpad_h = current_dim - pad_y
    unpad_w = current_dim - pad_x
    boxes[:, 0] = ((boxes[:, 0] - pad_x // 2) / unpad_w) * orig_w
    boxes[:, 1] = ((boxes[:, 1] - pad_y // 2) / unpad_h) * orig_h
    boxes[:, 2] = ((boxes[:, 2] - pad_x // 2) / unpad_w) * orig_w
    boxes[:, 3] = ((boxes[:, 3] - pad_y // 2) / unpad_h) * orig_h
    return boxes


def xywh2xyxy(x):
    y = x.new(x.shape)
    y[..., 0] = x[..., 0] - x[..., 2] / 2
    y[..., 1] = x[..., 1] - x[..., 3] / 2
    y[..., 2] = x[..., 0] + x[..., 2] / 2
    y[..., 3] = x[..., 1] + x[..., 3] / 2
    return y


def ap_per_class(tp, conf, pred_cls, target_cls):
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]
    unique_classes = np.unique(target_cls)
    ap, p, r = [], [], []
    for c in tqdm.tqdm(unique_classes, desc="Computing AP"):
        i = pred_cls == c
        n_gt = (target_cls == c).sum()  
        n_p = i.sum()

        if n_p == 0 and n_gt == 0:
            continue
        elif n_p == 0 or n_gt == 0:
            ap.append(0)
            r.append(0)
            p.append(0)
        else:
            fpc = (1 - tp[i]).cumsum()
            tpc = (tp[i]).cumsum()

            recall_curve = tpc / (n_gt + 1e-16)
            r.append(recall_curve[-1])

            precision_curve = tpc / (tpc + fpc)
            p.append(precision_curve[-1])

            ap.append(compute_ap(recall_curve, precision_curve))

    p, r, ap = np.array(p), np.array(r), np.array(ap)
    f1 = 2 * p * r / (p + r + 1e-16)

    return p, r, ap, f1, unique_classes.astype("int32")


def compute_ap(recall, precision):
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])
    i = np.where(mrec[1:] != mrec[:-1])[0]
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


def get_batch_statistics(outputs, targets, iou_threshold):
    batch_metrics = []
    for sample_i in range(len(outputs)):

        if outputs[sample_i] is None:
            continue

        output = outputs[sample_i]
        pred_boxes = output[:, :4]
        pred_scores = output[:, 4]
        pred_labels = output[:, -1]

        true_positives = np.zeros(pred_boxes.shape[0])

        annotations = targets[targets[:, 0] == sample_i][:, 1:]
        target_labels = annotations[:, 0] if len(annotations) else []
        if len(annotations):
            detected_boxes = []
            target_boxes = annotations[:, 1:]

            for pred_i, (pred_box, pred_label) in enumerate(zip(pred_boxes, pred_labels)):

                if len(detected_boxes) == len(annotations):
                    break
                if pred_label not in target_labels:
                    continue

                iou, box_index = bbox_iou(pred_box.unsqueeze(0), target_boxes).max(0)
                if iou >= iou_threshold and box_index not in detected_boxes:
                    true_positives[pred_i] = 1
                    detected_boxes += [box_index]
        batch_metrics.append([true_positives, pred_scores, pred_labels])
    return batch_metrics


def bbox_wh_iou(wh1, wh2):
    wh2 = wh2.t()
    w1, h1 = wh1[0], wh1[1]
    w2, h2 = wh2[0], wh2[1]
    inter_area = torch.min(w1, w2) * torch.min(h1, h2)
    union_area = (w1 * h1 + 1e-16) + w2 * h2 - inter_area
    return inter_area / union_area


def bbox_iou(box1, box2, x1y1x2y2=True):
    if not x1y1x2y2:
        b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
        b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
        b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
        b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2
    else:
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]

    inter_rect_x1 = torch.max(b1_x1, b2_x1)
    inter_rect_y1 = torch.max(b1_y1, b2_y1)
    inter_rect_x2 = torch.min(b1_x2, b2_x2)
    inter_rect_y2 = torch.min(b1_y2, b2_y2)
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * torch.clamp(
        inter_rect_y2 - inter_rect_y1 + 1, min=0
    )
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

    return iou


def non_max_suppression(prediction, conf_thres=0.5, nms_thres=0.4):
    prediction[..., :4] = xywh2xyxy(prediction[..., :4])
    output = [None for _ in range(len(prediction))]
    for image_i, image_pred in enumerate(prediction):
        image_pred = image_pred[image_pred[:, 4] >= conf_thres]
        if not image_pred.size(0):
            continue
        score = image_pred[:, 4] * image_pred[:, 5:].max(1)[0]
        image_pred = image_pred[(-score).argsort()]
        class_confs, class_preds = image_pred[:, 5:].max(1, keepdim=True)
        detections = torch.cat((image_pred[:, :5], class_confs.float(), class_preds.float()), 1)
        keep_boxes = []
        while detections.size(0):
            large_overlap = bbox_iou(detections[0, :4].unsqueeze(0), detections[:, :4]) > nms_thres
            label_match = detections[0, -1] == detections[:, -1]
            invalid = large_overlap & label_match
            weights = detections[invalid, 4:5]
            detections[0, :4] = (weights * detections[invalid, :4]).sum(0) / weights.sum()
            keep_boxes += [detections[0]]
            detections = detections[~invalid]
        if keep_boxes:
            output[image_i] = torch.stack(keep_boxes)

    return output


def build_targets(pred_boxes, pred_cls, target, anchors, ignore_thres):

    BoolTensor = torch.cuda.BoolTensor if pred_boxes.is_cuda else torch.BoolTensor
    FloatTensor = torch.cuda.FloatTensor if pred_boxes.is_cuda else torch.FloatTensor

    nB = pred_boxes.size(0)
    nA = pred_boxes.size(1)
    nC = pred_cls.size(-1)
    nG = pred_boxes.size(2)

    obj_mask = BoolTensor(nB, nA, nG, nG).fill_(0)
    noobj_mask = BoolTensor(nB, nA, nG, nG).fill_(1)
    class_mask = FloatTensor(nB, nA, nG, nG).fill_(0)
    iou_scores = FloatTensor(nB, nA, nG, nG).fill_(0)
    tx = FloatTensor(nB, nA, nG, nG).fill_(0)
    ty = FloatTensor(nB, nA, nG, nG).fill_(0)
    tw = FloatTensor(nB, nA, nG, nG).fill_(0)
    th = FloatTensor(nB, nA, nG, nG).fill_(0)
    tcls = FloatTensor(nB, nA, nG, nG, nC).fill_(0)

    target_boxes = target[:, 2:6] * nG
    gxy = target_boxes[:, :2]
    gwh = target_boxes[:, 2:]
    ious = torch.stack([bbox_wh_iou(anchor, gwh) for anchor in anchors])
    best_ious, best_n = ious.max(0)
    b, target_labels = target[:, :2].long().t()
    gx, gy = gxy.t()
    gw, gh = gwh.t()
    gi, gj = gxy.long().t()
    obj_mask[b, best_n, gj, gi] = 1
    noobj_mask[b, best_n, gj, gi] = 0

    for i, anchor_ious in enumerate(ious.t()):
        noobj_mask[b[i], anchor_ious > ignore_thres, gj[i], gi[i]] = 0

    tx[b, best_n, gj, gi] = gx - gx.floor()
    ty[b, best_n, gj, gi] = gy - gy.floor()
    tw[b, best_n, gj, gi] = torch.log(gw / anchors[best_n][:, 0] + 1e-16)
    th[b, best_n, gj, gi] = torch.log(gh / anchors[best_n][:, 1] + 1e-16)
    tcls[b, best_n, gj, gi, target_labels] = 1
    class_mask[b, best_n, gj, gi] = (pred_cls[b, best_n, gj, gi].argmax(-1) == target_labels).float()
    iou_scores[b, best_n, gj, gi] = bbox_iou(pred_boxes[b, best_n, gj, gi], target_boxes, x1y1x2y2=False)

    tconf = obj_mask.float()
    return iou_scores, class_mask, obj_mask, noobj_mask, tx, ty, tw, th, tcls, tconf

def horisontal_flip(images, targets):
    images = torch.flip(images, [-1])
    targets[:, 2] = 1 - targets[:, 2]
    return images, targets

#---------------------------------------------------
def pad_to_square(img, pad_value):
    c, h, w = img.shape
    dim_diff = np.abs(h - w)
    pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
    pad = (0, 0, pad1, pad2) if h <= w else (pad1, pad2, 0, 0)
    img = F.pad(img, pad, "constant", value=pad_value)
    return img, pad


def resize(image, size):
    image = F.interpolate(image.unsqueeze(0), size=size, mode="nearest").squeeze(0)
    return image


def random_resize(images, min_size=288, max_size=448):
    new_size = random.sample(list(range(min_size, max_size + 1, 32)), 1)[0]
    images = F.interpolate(images, size=new_size, mode="nearest")
    return images


class ImageFolder(Dataset):
    def __init__(self, folder_path, img_size=416):
        self.files = sorted(glob.glob("%s/*.*" % folder_path))
        self.img_size = img_size

    def __getitem__(self, index):
        img_path = self.files[index % len(self.files)]
        img = transforms.ToTensor()(Image.open(img_path))
        img, _ = pad_to_square(img, 0)
        img = resize(img, self.img_size)

        return img_path, img

    def __len__(self):
        return len(self.files)


class ListDataset(Dataset):
    def __init__(self, list_path, img_size=416, augment=True, multiscale=True, normalized_labels=True):
        with open(list_path, "r") as file:
            self.img_files = file.readlines()

        self.label_files = [
            path.replace("images", "labels").replace(".png", ".txt").replace(".jpg", ".txt")
            for path in self.img_files
        ]
        self.img_size = img_size
        self.max_objects = 100
        self.augment = augment
        self.multiscale = multiscale
        self.normalized_labels = normalized_labels
        self.min_size = self.img_size - 3 * 32
        self.max_size = self.img_size + 3 * 32
        self.batch_count = 0

    def __getitem__(self, index):
        img_path = self.img_files[index % len(self.img_files)].rstrip()
        img = transforms.ToTensor()(Image.open(img_path).convert('RGB'))
        if len(img.shape) != 3:
            img = img.unsqueeze(0)
            img = img.expand((3, img.shape[1:]))

        _, h, w = img.shape
        h_factor, w_factor = (h, w) if self.normalized_labels else (1, 1)
        img, pad = pad_to_square(img, 0)
        _, padded_h, padded_w = img.shape

        label_path = self.label_files[index % len(self.img_files)].rstrip()

        targets = None
        if os.path.exists(label_path):
            boxes = torch.from_numpy(np.loadtxt(label_path).reshape(-1, 5))
            # Extract coordinates for unpadded + unscaled image
            x1 = w_factor * (boxes[:, 1] - boxes[:, 3] / 2)
            y1 = h_factor * (boxes[:, 2] - boxes[:, 4] / 2)
            x2 = w_factor * (boxes[:, 1] + boxes[:, 3] / 2)
            y2 = h_factor * (boxes[:, 2] + boxes[:, 4] / 2)
            # Adjust for added padding
            x1 += pad[0]
            y1 += pad[2]
            x2 += pad[1]
            y2 += pad[3]
            # Returns (x, y, w, h)
            boxes[:, 1] = ((x1 + x2) / 2) / padded_w
            boxes[:, 2] = ((y1 + y2) / 2) / padded_h
            boxes[:, 3] *= w_factor / padded_w
            boxes[:, 4] *= h_factor / padded_h

            targets = torch.zeros((len(boxes), 6))
            targets[:, 1:] = boxes

        if self.augment:
            if np.random.random() < 0.5:
                img, targets = horisontal_flip(img, targets)

        return img_path, img, targets

    def collate_fn(self, batch):
        paths, imgs, targets = list(zip(*batch))
        targets = [boxes for boxes in targets if boxes is not None]
        for i, boxes in enumerate(targets):
            boxes[:, 0] = i
        targets = torch.cat(targets, 0)
        if self.multiscale and self.batch_count % 10 == 0:
            self.img_size = random.choice(range(self.min_size, self.max_size + 1, 32))
        imgs = torch.stack([resize(img, self.img_size) for img in imgs])
        self.batch_count += 1
        return paths, imgs, targets

    def __len__(self):
        return len(self.img_files)

# import MODELOS
def parse_model_config(path):
    file = open(path, 'r')
    lines = file.read().split('\n')
    lines = [x for x in lines if x and not x.startswith('#')]
    lines = [x.rstrip().lstrip() for x in lines]
    module_defs = []
    for line in lines:
        if line.startswith('['): 
            module_defs.append({})
            module_defs[-1]['type'] = line[1:-1].rstrip()
            if module_defs[-1]['type'] == 'convolutional':
                module_defs[-1]['batch_normalize'] = 0
        else:
            key, value = line.split("=")
            value = value.strip()
            module_defs[-1][key.rstrip()] = value.strip()

    return module_defs

def parse_data_config(path):
    options = dict()
    options['gpus'] = '0,1,2,3'
    options['num_workers'] = '10'
    with open(path, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.strip()
        if line == '' or line.startswith('#'):
            continue
        key, value = line.split('=')
        options[key.strip()] = value.strip()
    return options

def create_modules(module_defs):
    hyperparams = module_defs.pop(0)
    output_filters = [int(hyperparams["channels"])]
    module_list = nn.ModuleList()
    for module_i, module_def in enumerate(module_defs):
        modules = nn.Sequential()

        if module_def["type"] == "convolutional":
            bn = int(module_def["batch_normalize"])
            filters = int(module_def["filters"])
            kernel_size = int(module_def["size"])
            pad = (kernel_size - 1) // 2
            modules.add_module(
                f"conv_{module_i}",
                nn.Conv2d(
                    in_channels=output_filters[-1],
                    out_channels=filters,
                    kernel_size=kernel_size,
                    stride=int(module_def["stride"]),
                    padding=pad,
                    bias=not bn,
                ),
            )
            if bn:
                modules.add_module(f"batch_norm_{module_i}", nn.BatchNorm2d(filters, momentum=0.9, eps=1e-5))
            if module_def["activation"] == "leaky":
                modules.add_module(f"leaky_{module_i}", nn.LeakyReLU(0.1))

        elif module_def["type"] == "maxpool":
            kernel_size = int(module_def["size"])
            stride = int(module_def["stride"])
            if kernel_size == 2 and stride == 1:
                modules.add_module(f"_debug_padding_{module_i}", nn.ZeroPad2d((0, 1, 0, 1)))
            maxpool = nn.MaxPool2d(kernel_size=kernel_size, stride=stride, padding=int((kernel_size - 1) // 2))
            modules.add_module(f"maxpool_{module_i}", maxpool)

        elif module_def["type"] == "upsample":
            upsample = Upsample(scale_factor=int(module_def["stride"]), mode="nearest")
            modules.add_module(f"upsample_{module_i}", upsample)

        elif module_def["type"] == "route":
            layers = [int(x) for x in module_def["layers"].split(",")]
            filters = sum([output_filters[1:][i] for i in layers])
            modules.add_module(f"route_{module_i}", EmptyLayer())

        elif module_def["type"] == "shortcut":
            filters = output_filters[1:][int(module_def["from"])]
            modules.add_module(f"shortcut_{module_i}", EmptyLayer())

        elif module_def["type"] == "yolo":
            anchor_idxs = [int(x) for x in module_def["mask"].split(",")]
            # Extract anchors
            anchors = [int(x) for x in module_def["anchors"].split(",")]
            anchors = [(anchors[i], anchors[i + 1]) for i in range(0, len(anchors), 2)]
            anchors = [anchors[i] for i in anchor_idxs]
            num_classes = int(module_def["classes"])
            img_size = int(hyperparams["height"])
            # Define detection layer
            yolo_layer = YOLOLayer(anchors, num_classes, img_size)
            modules.add_module(f"yolo_{module_i}", yolo_layer)
        # Register module list and number of output filters
        module_list.append(modules)
        output_filters.append(filters)

    return hyperparams, module_list


class Upsample(nn.Module):
    def __init__(self, scale_factor, mode="nearest"):
        super(Upsample, self).__init__()
        self.scale_factor = scale_factor
        self.mode = mode

    def forward(self, x):
        x = F.interpolate(x, scale_factor=self.scale_factor, mode=self.mode)
        return x


class EmptyLayer(nn.Module):
    def __init__(self):
        super(EmptyLayer, self).__init__()


class YOLOLayer(nn.Module):
    def __init__(self, anchors, num_classes, img_dim=416):
        super(YOLOLayer, self).__init__()
        self.anchors = anchors
        self.num_anchors = len(anchors)
        self.num_classes = num_classes
        self.ignore_thres = 0.5
        self.mse_loss = nn.MSELoss()
        self.bce_loss = nn.BCELoss()
        self.obj_scale = 1
        self.noobj_scale = 100
        self.metrics = {}
        self.img_dim = img_dim
        self.grid_size = 0 

    def compute_grid_offsets(self, grid_size, cuda=True):
        self.grid_size = grid_size
        g = self.grid_size
        FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
        self.stride = self.img_dim / self.grid_size
        self.grid_x = torch.arange(g).repeat(g, 1).view([1, 1, g, g]).type(FloatTensor)
        self.grid_y = torch.arange(g).repeat(g, 1).t().view([1, 1, g, g]).type(FloatTensor)
        self.scaled_anchors = FloatTensor([(a_w / self.stride, a_h / self.stride) for a_w, a_h in self.anchors])
        self.anchor_w = self.scaled_anchors[:, 0:1].view((1, self.num_anchors, 1, 1))
        self.anchor_h = self.scaled_anchors[:, 1:2].view((1, self.num_anchors, 1, 1))

    def forward(self, x, targets=None, img_dim=None):
        FloatTensor = torch.cuda.FloatTensor if x.is_cuda else torch.FloatTensor
        LongTensor = torch.cuda.LongTensor if x.is_cuda else torch.LongTensor
        ByteTensor = torch.cuda.ByteTensor if x.is_cuda else torch.ByteTensor

        self.img_dim = img_dim
        num_samples = x.size(0)
        grid_size = x.size(2)

        prediction = (
            x.view(num_samples, self.num_anchors, self.num_classes + 5, grid_size, grid_size)
            .permute(0, 1, 3, 4, 2)
            .contiguous()
        )

        x = torch.sigmoid(prediction[..., 0])  
        y = torch.sigmoid(prediction[..., 1])  
        w = prediction[..., 2]  
        h = prediction[..., 3]  
        pred_conf = torch.sigmoid(prediction[..., 4])  
        pred_cls = torch.sigmoid(prediction[..., 5:])  

        if grid_size != self.grid_size:
            self.compute_grid_offsets(grid_size, cuda=x.is_cuda)

        pred_boxes = FloatTensor(prediction[..., :4].shape)
        pred_boxes[..., 0] = x.data + self.grid_x
        pred_boxes[..., 1] = y.data + self.grid_y
        pred_boxes[..., 2] = torch.exp(w.data) * self.anchor_w
        pred_boxes[..., 3] = torch.exp(h.data) * self.anchor_h

        output = torch.cat(
            (
                pred_boxes.view(num_samples, -1, 4) * self.stride,
                pred_conf.view(num_samples, -1, 1),
                pred_cls.view(num_samples, -1, self.num_classes),
            ),
            -1,
        )

        if targets is None:
            return output, 0
        else:
            iou_scores, class_mask, obj_mask, noobj_mask, tx, ty, tw, th, tcls, tconf = build_targets(
                pred_boxes=pred_boxes,
                pred_cls=pred_cls,
                target=targets,
                anchors=self.scaled_anchors,
                ignore_thres=self.ignore_thres,
            )

            loss_x = self.mse_loss(x[obj_mask], tx[obj_mask])
            loss_y = self.mse_loss(y[obj_mask], ty[obj_mask])
            loss_w = self.mse_loss(w[obj_mask], tw[obj_mask])
            loss_h = self.mse_loss(h[obj_mask], th[obj_mask])
            loss_conf_obj = self.bce_loss(pred_conf[obj_mask], tconf[obj_mask])
            loss_conf_noobj = self.bce_loss(pred_conf[noobj_mask], tconf[noobj_mask])
            loss_conf = self.obj_scale * loss_conf_obj + self.noobj_scale * loss_conf_noobj
            loss_cls = self.bce_loss(pred_cls[obj_mask], tcls[obj_mask])
            total_loss = loss_x + loss_y + loss_w + loss_h + loss_conf + loss_cls

            cls_acc = 100 * class_mask[obj_mask].mean()
            conf_obj = pred_conf[obj_mask].mean()
            conf_noobj = pred_conf[noobj_mask].mean()
            conf50 = (pred_conf > 0.5).float()
            iou50 = (iou_scores > 0.5).float()
            iou75 = (iou_scores > 0.75).float()
            detected_mask = conf50 * class_mask * tconf
            precision = torch.sum(iou50 * detected_mask) / (conf50.sum() + 1e-16)
            recall50 = torch.sum(iou50 * detected_mask) / (obj_mask.sum() + 1e-16)
            recall75 = torch.sum(iou75 * detected_mask) / (obj_mask.sum() + 1e-16)

            self.metrics = {
                "loss": to_cpu(total_loss).item(),
                "x": to_cpu(loss_x).item(),
                "y": to_cpu(loss_y).item(),
                "w": to_cpu(loss_w).item(),
                "h": to_cpu(loss_h).item(),
                "conf": to_cpu(loss_conf).item(),
                "cls": to_cpu(loss_cls).item(),
                "cls_acc": to_cpu(cls_acc).item(),
                "recall50": to_cpu(recall50).item(),
                "recall75": to_cpu(recall75).item(),
                "precision": to_cpu(precision).item(),
                "conf_obj": to_cpu(conf_obj).item(),
                "conf_noobj": to_cpu(conf_noobj).item(),
                "grid_size": grid_size,
            }

            return output, total_loss


class Darknet(nn.Module):
    def __init__(self, config_path, img_size=416):
        super(Darknet, self).__init__()
        self.module_defs = parse_model_config(config_path)
        self.hyperparams, self.module_list = create_modules(self.module_defs)
        self.yolo_layers = [layer[0] for layer in self.module_list if hasattr(layer[0], "metrics")]
        self.img_size = img_size
        self.seen = 0
        self.header_info = np.array([0, 0, 0, self.seen, 0], dtype=np.int32)

    def forward(self, x, targets=None):
        img_dim = x.shape[2]
        loss = 0
        layer_outputs, yolo_outputs = [], []
        for i, (module_def, module) in enumerate(zip(self.module_defs, self.module_list)):
            if module_def["type"] in ["convolutional", "upsample", "maxpool"]:
                x = module(x)
            elif module_def["type"] == "route":
                x = torch.cat([layer_outputs[int(layer_i)] for layer_i in module_def["layers"].split(",")], 1)
            elif module_def["type"] == "shortcut":
                layer_i = int(module_def["from"])
                x = layer_outputs[-1] + layer_outputs[layer_i]
            elif module_def["type"] == "yolo":
                x, layer_loss = module[0](x, targets, img_dim)
                loss += layer_loss
                yolo_outputs.append(x)
            layer_outputs.append(x)
        yolo_outputs = to_cpu(torch.cat(yolo_outputs, 1))
        return yolo_outputs if targets is None else (loss, yolo_outputs)

    def load_darknet_weights(self, weights_path):
        with open(weights_path, "rb") as f:
            header = np.fromfile(f, dtype=np.int32, count=5)  
            self.seen = header[3]  
            weights = np.fromfile(f, dtype=np.float32)

        cutoff = None
        if "darknet53.conv.74" in weights_path:
            cutoff = 75

        ptr = 0
        for i, (module_def, module) in enumerate(zip(self.module_defs, self.module_list)):
            if i == cutoff:
                break
            if module_def["type"] == "convolutional":
                conv_layer = module[0]
                if module_def["batch_normalize"]:
                    bn_layer = module[1]
                    num_b = bn_layer.bias.numel()
                    
                    bn_b = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(bn_layer.bias)
                    bn_layer.bias.data.copy_(bn_b)
                    ptr += num_b
                    bn_w = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(bn_layer.weight)
                    bn_layer.weight.data.copy_(bn_w)
                    ptr += num_b
                    bn_rm = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(bn_layer.running_mean)
                    bn_layer.running_mean.data.copy_(bn_rm)
                    ptr += num_b
                    bn_rv = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(bn_layer.running_var)
                    bn_layer.running_var.data.copy_(bn_rv)
                    ptr += num_b
                else:
                    num_b = conv_layer.bias.numel()
                    conv_b = torch.from_numpy(weights[ptr : ptr + num_b]).view_as(conv_layer.bias)
                    conv_layer.bias.data.copy_(conv_b)
                    ptr += num_b
                num_w = conv_layer.weight.numel()
                conv_w = torch.from_numpy(weights[ptr : ptr + num_w]).view_as(conv_layer.weight)
                conv_layer.weight.data.copy_(conv_w)
                ptr += num_w

    def save_darknet_weights(self, path, cutoff=-1):
        fp = open(path, "wb")
        self.header_info[3] = self.seen
        self.header_info.tofile(fp)

        for i, (module_def, module) in enumerate(zip(self.module_defs[:cutoff], self.module_list[:cutoff])):
            if module_def["type"] == "convolutional":
                conv_layer = module[0]
                if module_def["batch_normalize"]:
                    bn_layer = module[1]
                    bn_layer.bias.data.cpu().numpy().tofile(fp)
                    bn_layer.weight.data.cpu().numpy().tofile(fp)
                    bn_layer.running_mean.data.cpu().numpy().tofile(fp)
                    bn_layer.running_var.data.cpu().numpy().tofile(fp)
                else:
                    conv_layer.bias.data.cpu().numpy().tofile(fp)
                conv_layer.weight.data.cpu().numpy().tofile(fp)

        fp.close()


# Class Cuadrante "CuadranteRalec"
Clase Cuadrante de verificación de pertenencia y complementación en función de algún cuadrante encontrado, completando la cantidad de cuadrantes no encontrados por la Red Convolucional YOLO.


![Funcion de pertenencia del cuadrante](http://drive.google.com/uc?export=view&id=1Cys0_Vr9F8zOhYoPa0nrpAZfMrtwFkCj)

In [ ]:
class cuadrante:
    lista=[]
    cuadrante=[]
    p=[]
    h_cu=0
    w_cu=0
    def __init__(self,csi,csd,cii,cid,h_c,w_c):
        self.h_cu=h_c
        self.w_cu=w_c
        self.cuadrante=np.zeros(4,dtype=int)
        self.lista = np.zeros(16,dtype=int)

        if len(csi)==4 and self.correcto(csi[3],csi[1])==1:
            self.cuadrante[0]=1
            self.lista[0] = csi[2]  # y1
            self.lista[1] = csi[3]  # y2
            self.lista[2] = csi[0]  # x1
            self.lista[3] = csi[1]  # x2
        if len(csd)==4 and self.correcto(csd[3],csd[1])==2:
            self.cuadrante[1]=1
            self.lista[4] = csd[2]  # y1
            self.lista[5] = csd[3]  # y2
            self.lista[6] = csd[0]  # x1
            self.lista[7] = csd[1]  # x2
        if len(cii)==4 and self.correcto(cii[3],cii[1])==3:
            self.cuadrante[2]=1
            self.lista[8] = cii[2]  # y1
            self.lista[9] = cii[3]  # y2
            self.lista[10] = cii[0]  # x1
            self.lista[11] = cii[1]  # x2
        if len(cid)==4 and self.correcto(cid[3],cid[1])==4:
            self.cuadrante[3]=1
            self.lista[12] = cid[2]  # y1
            self.lista[13] = cid[3]  # y2
            self.lista[14] = cid[0]  # x1
            self.lista[15] = cid[1]  # x2

    def correcto(self,h,w):
        sector=0
        #cuadrante 1
        if ((h-self.h_cu/4)<self.h_cu/2) and ((w-self.w_cu/4)<self.w_cu/2):
            sector=1
        #cuadrante 2
        if ((h-self.h_cu/4)<self.h_cu/2) and ((w-self.w_cu/4)>self.w_cu/2):
            sector=2
        #cuadrante 3
        if((h-self.h_cu/4)>self.h_cu/2) and ((w-self.w_cu/4)<self.w_cu/2):
            sector=3
        # cuadrante 4
        if ((h - self.h_cu / 4) > self.h_cu / 2) and ((w - self.w_cu / 4) < self.w_cu / 2):
            sector=4

        return sector

    def completo(self):
        return (np.prod(self.cuadrante),self.cuadrante)

    def cuadranteCenter(self,index):
        y=0
        x=0
        if index==0:
            y = int((self.h_cu + (2 * self.lista[1])) / 4)
            x = int((self.w_cu + (2 * self.lista[3])) / 4)
        if index==1:
            y = int((self.h_cu + (2 * self.lista[5])) / 4)
            x = int((self.w_cu + (2 * self.lista[6])) / 4)
        if index==2:
            y = int((self.h_cu + (2 * self.lista[8])) / 4)
            x = int((self.w_cu + (2 * self.lista[11])) / 4)
        if index==3:
            y = int((self.h_cu + (2 * self.lista[12])) / 4)
            x = int((self.w_cu + (2 * self.lista[14])) / 4)
        temp = [y, x]
        self.p = temp

    def cuadranteCompletar(self,x,y):
        # en funcion de Cuadrante 0
        if x==0 and y==1:
            self.lista[4] = self.lista[0]
            self.lista[5] = self.lista[1]
            self.lista[6] = self.p[1] - 100
            self.lista[7] = int((self.w_cu + (2 * (self.lista[6] + self.lista[3] + self.lista[2]))) / 4)

        if x==0 and y==2:
            self.lista[8] = self.p[0] - 200
            self.lista[9] = int((self.h_cu / 2) + self.lista[8])
            c = 0
            if (self.w_cu / 2) - self.lista[2] > 300:
                c = -200
            self.lista[10] = self.lista[2] + c
            self.lista[11] = self.p[1] + 200

        if x==0 and y==3:
            self.lista[12] = self.p[0] - 100
            self.lista[13] = int((self.h_cu / 2) + self.lista[12])
            self.lista[14] = self.p[1] - 100
            self.lista[15] = int((self.w_cu / 2) + self.lista[14])
        # en funcion de Cuadrante 1 csd
        if x==1 and y==0:
            self.lista[0] = self.lista[4]
            self.lista[1] = self.lista[5]
            self.lista[3] = self.p[1] + 100
            self.lista[2] = int((2 * (self.lista[7] - self.lista[6] - self.lista[3]) + self.w_cu) / 4)

        if x==1 and y==2:
            self.lista[8] = self.p[0] - 100
            self.lista[9] = int((self.h_cu / 2) + self.lista[8]) - 200
            c = 0
            if (self.w_cu / 2) - self.lista[2] > 300:
                c = -200
            self.lista[10] = self.lista[2] + c
            # self.lista[10] =int((2*(self.lista[7]+self.lista[6]+self.lista[3])+self.w)/4)
            self.lista[11] = self.p[1] + 100

        if x==1 and y==3:
            self.lista[12] = self.p[0] - 200
            self.lista[13] = int((self.h_cu / 2) + self.lista[12]) - 200
            self.lista[14] = self.p[1] - 200
            c = 0
            if (self.w_cu / 2) - self.lista[14] > 300:
                c = -200
            self.lista[15] = int((self.w_cu / 2) + self.lista[14]) + c

        # en funcion de Cuadrante 2 cii
        if x==2 and y==0:
            self.lista[1] = self.p[0] - 100
            c = 0
            t = int((2 * (self.lista[9] - self.lista[8] - self.lista[1]) + self.h_cu) / 4)
            if t > 500:
                c = -300
            self.lista[0] = t + c
            c = 0
            if ((self.w_cu / 2) - self.lista[10] > 300):
                c = -100
            self.lista[2] = self.lista[10] - c
            self.lista[3] = self.lista[11]

        if x==2 and y==1:
            self.lista[4] = self.lista[0]
            self.lista[5] = self.p[0] + 100
            self.lista[6] = self.p[1] - 100
            self.lista[7] = int((self.w_cu / 2) + self.lista[6]) - 100

        if x==2 and y==3:
            self.lista[12] = self.lista[8]
            self.lista[13] = self.lista[9]
            self.lista[14] = self.p[1] - 200
            self.lista[15] = int((self.w_cu / 2) + self.lista[14])

        # en funcion de Cuadrante 3 cid
        if x==3 and y==0:
            self.lista[1] = self.p[0] + 100
            self.lista[3] = self.p[1] + 100
            t = int((2 * (self.lista[13] - self.lista[12] - self.lista[1]) + self.h_cu) / 4)
            c = 0
            if t - 200 > 0:
                c = -200
            self.lista[0] = t + c
            t = int((2 * (self.lista[15] - self.lista[14] - self.lista[3]) + self.w_cu) / 4)
            c = 0
            if t - 300 > 0:
                c + -300
            self.lista[2] = t + c

        if x==3 and y==1:
            self.lista[5] = self.lista[1] - 100
            self.lista[6] = self.lista[14]
            self.lista[7] = self.lista[15]
            # self.lista[4]=int((2*(self.lista[13]-self.lista[12]-self.lista[5])+self.h)/4)
            self.lista[4] = self.lista[0]

        if x==3 and y==2:
            self.lista[8] = self.lista[12]
            self.lista[9] = self.lista[13]
            self.lista[11] = self.p[1] + 100
            # self.lista[10] =int((2*(self.lista[15]-self.lista[14]-self.lista[11])+self.w)/4)
            c = 0
            if self.lista[2] - 200 > 0:
                c = -200
            self.lista[10] = self.lista[2] + c

    def getLista(self):
        completo=np.prod(self.cuadrante)
        if completo == 0:
            for l in range(0, len(self.cuadrante)):
                if self.cuadrante[l] == 1 and completo == 0:
                    self.cuadranteCenter(l)
                    for index in range(0, len(self.cuadrante)):
                        if l != index and self.cuadrante[index] == 0:
                            self.cuadranteCompletar(l, index)
                            self.cuadrante[index] = 1
                completo = np.prod(self.cuadrante)

        return self.lista

# Funciones y Procedimientos
Funciones y procedimientos para la deteccion del cuadrante, en su etapa de identificacion:

**DcmToJpg:** realiza la lectura y verifica si la Radiografia de Torax PosteroAnterio es soportada por la libreria pydicom o si cuenta con la cantidad de bit's necesaria para poder ser tratada. el archivo **.dcm**  es guardado en el directorio **./torax/nombreDicom.jpg**.

**detectorCuadrante:** detecta la pertenencia de un cuadrante de acuerdo a las siguientes etiquetas:
*     Jcsi: Cuadrante Superior Izquierdo
*     Jcsd: Cuadrante Superior Derecho
*     Jcii: Cuadrante Inferior Izquierdo
*     Jcid: Cuadrante Inferior Derecho

los cuadrantes detectados se guardan en **./torax/cuadrante/nombreDicom/nombreDicom_jc__.jpg**

In [ ]:
def convertiRgb(img):
    b=img[:, :, 0].copy()
    g=img[:, :, 1].copy()
    rc=img[:, :, 2].copy()

    img[:, :, 0]=rc
    img[:, :, 1] =g
    img[:, :, 2] =b

    return img

def convertirBgr(img):
    rc = img[:, :, 0].copy()
    g = img[:, :, 1].copy()
    b = img[:, :, 2].copy()
    
    img[:, :, 0] = b
    img[:, :, 1] = g
    img[:, :, 2] = rc
    
    return img

def desvorde(lista_d):
    for i in range(0,len(lista_d)):
        if(lista_d[i]<0):
            lista_d[i]=10
    return lista_d

def DcmToJpg(pathDcm):
    ds=pydi.dcmread(pathDcm)
    file=''
    temp_=''
    name=''
    try:
        d=np.array(ds.pixel_array)
        name=pathDcm[64:]
        file=name.replace('.dcm','')
        name=name.replace('.dcm','.jpg')
        temp_='./torax/cuadrante/'+file
        if not os.path.exists(temp_):
            os.mkdir(temp_)
        plt.imsave('./torax/'+name,d)
    except:
        print('Error Archivo DCM no Soportado')
    
    return(temp_,name,file)

def detectorCuadrante(ImgFile,ruta,name):
    jcsi_=[]
    jcsd_=[]
    jcii_=[]
    jcid_=[]
    lista_=[]
    cont=0

    imgRadio=cv2.imread(ImgFile)
    h_c=imgRadio.shape[0]
    w_c=imgRadio.shape[1]
    RGBimg=convertiRgb(imgRadio)
    imgTensor=transforms.ToTensor()(imgRadio)
    imgTensor,_=pad_to_square(imgTensor,0)
    imgTensor=resize(imgTensor,416)
    imgTensor=imgTensor.unsqueeze(0)
    imgTensor=Variable(imgTensor.type(Tensor))

    detections=modelo(imgTensor)
    detections=non_max_suppression(detections,confThres,0.4)
    
    for detection in detections:
        if detection is not None:
            detection=rescale_boxes(detection,416, RGBimg.shape[:2])
            for (x1,y1,x2,y2,conf,cls_conf,cls_pred)in detection:
                temp=[int(x1),int(x2),int(y1),int(y2)]
                if int(cls_pred)==0:
                    jcsi_=temp
                if int(cls_pred)==1:
                    jcsd_=temp
                if int(cls_pred)==2:
                    jcii_=temp
                if int(cls_pred)==3:
                    jcid_=temp

        jcsi_=desvorde(jcsi_)
        jcsd_=desvorde(jcsd_)
        jcii_=desvorde(jcii_)
        jcid_=desvorde(jcid_)

        r_lista=cuadrante(jcsi_,jcsd_,jcii_,jcid_,h_c,w_c)
        lista_=r_lista.getLista()
        lista_=desvorde(lista_)

        obj1=imgRadio[lista_[0]:lista_[1],lista_[2]:lista_[3]]
        obj2=imgRadio[lista_[4]:lista_[5],lista_[6]:lista_[7]]
        obj3=imgRadio[lista_[8]:lista_[9],lista_[10]:lista_[11]]
        obj4=imgRadio[lista_[12]:lista_[13],lista_[14]:lista_[15]]
        
        suma=np.sum(lista_)
        if suma>0:
            plt.imsave(ruta+'/'+name+'jcsi.jpg',obj1)
            plt.imsave(ruta+'/'+name+'jcsd.jpg',obj2)
            plt.imsave(ruta+'/'+name+'jcii.jpg',obj3)
            plt.imsave(ruta+'/'+name+'jcid.jpg',obj4)
        else:
            print(cont,'Error cuadrante no detectado')

# Lectura de Modelo YOLO y lectura de file test
* **weightPah:** pesos que permite reconocer los cuadrantes.
* **modelDef:** modelo YOLO. 
* **classPath:** numero de clases para la deteccion de los cuadrantes.
* **confThers:** porcentaje de validacion para la deteccion de un cuadrante

lectura de la carpeta test para la prediccion de Covid-19 del set de Datos **siim-codiv19-detection**, ruta **../input/siim-covid19-detection/test/**

In [ ]:
weightPath='../input/dataaug/yolov3_ckpt_299.pth'
modelDef='../input/datamodelo/yolov3-custom.cfg'
classPath='../input/datamodelo/classes.names'
confThres=0.90

pathTest='../input/siim-covid19-detection/test/'
temp=r'**/*.dcm'
l=list(pathlib.Path(pathTest).glob(temp))

path='./torax/'
if not os.path.exists(path):
    os.mkdir(path)
    os.mkdir(path+'cuadrante')

print('Segmento Terminado')

# Construccion Red YOLO
mediante torch selecionamos el dispocitivo de calculo y definimos el tensor de calculo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo=Darknet(modelDef,img_size=512).to(device)
modelo.load_state_dict(torch.load(weightPath))
modelo.eval()

classes=load_classes(classPath)
Tensor=torch.FloatTensor

print('Segmento Terminado')

# Proceso de Imagen DCM
**DcmToJpg:** convierte el archivo **.DCM** a **.JPG**., la funcion debuleve los datos ruta, archivo, name de la cual se obtiene la ruta del file.jpg, el file.jpg, y el nombre de la carpeta.

**detectorCuadrante:** detectamos el cuadrante por medio de la funcion, mandando como parametros imagen de origen, ruta de los cuadrantes, y el nombre. **detectorCuadrante('./torax/'+archivo,ruta,name)**


In [ ]:
#ruta,archivo,name=DcmToJpg('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
#ruta,archivo,name=DcmToJpg('../input/siim-covid19-detection/train/0159c2d40b52/e68dcd68c9b7/e498491b1526.dcm')
#ruta,archivo,name=DcmToJpg('../input/siim-covid19-detection/test/05ce6249b5c0/4f28dfe3ff88/e226747a2d76.dcm')
#ruta,archivo,name=DcmToJpg('../input/siim-covid19-detection/train/fab641b3ce7c/82fb75bee506/c2d72bbff9fc.dcm')
ruta,archivo,name=DcmToJpg('../input/siim-covid19-detection/test/c85581436d7c/230faa37bfd9/9f6cc74b0062.dcm')

print(ruta,archivo, name)

print('Segmento Terminado')

In [ ]:
detectorCuadrante('./torax/'+archivo,ruta,name)
print('Segmento Terminado')

# Proceso de Deteccion de Neumonía Atípica "Covid-19"

cargamos los pesos y el modelo en **ConvRalec**, [**modeloRalec.h5**, **pesosRalec.h5**] 

creamos el arreglo **rale=[['jcsi',0],['jcsd',0],['jcii',0],['jcid',0]]** para la clasificacion y propiedad del cuadrante.

redimencionamos las imagenes **altura=224** y **longitud=224**

clasificamos los cuadrantes y mostramos el resultado: sumamos los valores rale, la sumatoria al dar 0 da como resultado el **negativo al Covid-19**



In [ ]:
h_r,w_r=224,224
modelo_r='../input/modeloralec/modeloRalec.h5'
pesos_r='../input/modeloralec/pesosRalec.h5'
ConvRalec=load_model(modelo_r)
ConvRalec.load_weights(pesos_r)

pathC='./torax/cuadrante/'+name+'/'
rale=[['jcsi',0],['jcsd',0],['jcii',0],['jcid',0]]

print('Segmento Terminado')

Recorremos los cuadrantes y predecimos cada uno de ellos con el modelo convRalec

In [ ]:
for cuadrantes in os.listdir(pathC):
    temp_c=load_img(pathC+cuadrantes,target_size=(w_r,h_r))
    temp_c=img_to_array(temp_c)
    temp_c=np.expand_dims(temp_c,axis=0)
    tmp_c=ConvRalec.predict(temp_c)
    resulT=tmp_c[0]
    rx=np.argmax(resulT)
    if rx==0 or rx==1:
        if rx==1:
            rale[3][1]=1
    if rx==2 or rx==3:
        if rx==3:
            rale[2][1]=1
    if rx==4 or rx==5:
        if rx==5:
            rale[1][1]=1
    if rx==6 or rx==7:
        if rx==7:
            rale[0][1]=1

diag=0
for ra in rale:
    diag+=ra[1]
titulos=['Negatigo Covid-19','Negativo Covid-19','Negativo Covid-19','Negativo Covid-19']
if diag>0:
    print('=============================================')
    print('Resultados de Diagnostico')
    print('Paciente con Diagnostico Atipico')
    print('=============================================')
    print('Detalle Cuadrantes')
    i=0
    for ra in rale:
        if ra[1]==1:
            print('Cuadrante ',ra[0],'Atipico')
            titulos[i]='Atipico'
        else:
            print('Cuadrante ',ra[0],'Negativo')
        i+=1
else:
    print('Paciente con Diagnostico de Covid-19 Negativo')

# Cuadrantes y Resultados
Mostramos la **Radiografia de Torax PosteroAnterior** con sus respectivos cuadrantes con el detalle de su clasificacion. 

In [ ]:
imgRadio=cv2.imread('./torax/'+archivo)
imgRadio=convertirBgr(imgRadio)

plt.rcParams["figure.figsize"] = (12, 8)
plt.suptitle('Resultados de predicción ConvRalec',fontsize=20,ha='center')
plt.subplot2grid((2,3),(0,0),rowspan=2)
#plt.subplot(2,3,1)
plt.imshow(imgRadio)
plt.title('Radiografia de Torax AP')

obj1 = cv2.imread(ruta+'/'+name+'jcsi.jpg')
obj1=convertirBgr(obj1)
plt.subplot2grid((2, 3), (0, 1))
plt.imshow(obj1)
plt.title(titulos[0]+' JCSI')

obj2 = cv2.imread(ruta+'/'+name+'jcsd.jpg')
obj2=convertirBgr(obj2)
plt.subplot2grid((2, 3), (0, 2))
plt.imshow(obj2)
plt.title(titulos[1]+' JCSD')

obj3 = cv2.imread(ruta+'/'+name+'jcii.jpg')
obj3=convertirBgr(obj3)
plt.subplot2grid((2,3),(1,1))
plt.imshow(obj3)
plt.title(titulos[2]+' JCII')

obj4 = cv2.imread(ruta+'/'+name+'jcid.jpg')
obj4=convertirBgr(obj4)
plt.subplot2grid((2,3),(1,2))
plt.imshow(obj4)
plt.title(titulos[3]+' JCID')

plt.savefig('./torax/'+name+'dia.jpg',)
plt.show()


# Extracción de Características.
Lectura de los conjuntos de datos para la extracción de las características en la cuarta capa convolucionada y de maxpooling.  

In [ ]:
h_e,w_e=224,224
batch_size=16
path_Train='../input/dataconvralec/torax/train/'
path_Valid='../input/dataconvralec/torax/valid/'
path_Test='../input/dataconvralec/torax/test/'

DataGenTrain=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
imaTrain=DataGenTrain.flow_from_directory(path_Train,target_size=(h_e,w_e),batch_size=batch_size,class_mode='categorical')

DataGenValid=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
imaValid=DataGenValid.flow_from_directory(path_Valid,target_size=(h_e,w_e),batch_size=batch_size,class_mode='categorical')

DataGenTest=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
imaTest=DataGenTest.flow_from_directory(path_Test,target_size=(h_e,w_e),batch_size=batch_size,class_mode='categorical')

print('Segmento Terminado')

Lectura de imagen, como parámetro la ruta del cuadrante detectado y su nombre "los cuadrantes deben de ser creados para obtener las características" 

In [ ]:
#imagen=tf.keras.preprocessing.image.load_img('./torax/cuadrante/c2d72bbff9fc/c2d72bbff9fcjcsi.jpg',target_size=(h_e,w_e))

imagen=tf.keras.preprocessing.image.load_img(ruta+'/'+name+'jcsi.jpg',target_size=(h_e,w_e))
imagen=tf.keras.preprocessing.image.img_to_array(imagen)
img=np.expand_dims(imagen,axis=0)
img.shape
print('Segmento Terminado')

Obtención de las gráficas de cada característica en el lote de imágenes espaciales en la cuarta capa, proceso de ingreso al flatening, para la predicción y detección de Neumonía Atípica.

In [ ]:
from collections import OrderedDict
capas=OrderedDict()
i=0
for capa in ConvRalec.layers:
    capas[capa.name]=capa

def get_activacion(modelo,capa,imagen):
    f=tf.keras.backend.function([modelo.layers[0].input],[capa.output])
    activacion=f((imagen))
    return activacion

capa_nombre='conv_4'
#capa_nombre='relu_4'
#capa_nombre='max_4'
capa=capas[capa_nombre]
activador=get_activacion(ConvRalec,capa,img)
activo=activador[0]
img_activo=activo[0]

fig=plt.figure(figsize=(20,20))
n=8

for i in range(8):
    for j in range(8):
        idx=n*i+j
        ax=fig.add_subplot(n,n,idx+1)
        ax.imshow(img_activo[:,:,idx])

# Matriz de Confusión.

Función de Matriz de confusión, recorre los datos de entrenamiento (DE), validación (DV) y test (DT), los mismos son clasificados en el arreglo **"resulT=['jcidn','jcidp','jciin','jciip','jcsdn','jcsdp','jcsin','jcsip']"**, para contabilizar las predicciones positivas y negativas.

![array de resultado de prediccion](http://drive.google.com/uc?export=view&id=1IdL-ITFptn6Iummq2_LNNqtZu-68nyUV)

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

def confucion(store):
    k=-1
    lTest=[]
    lPred=[]
    path=path_Test
    c=['jcidn','jcidp','jciin','jciip','jcsdn','jcsdp','jcsin','jcsip']
    #c=['n','p']
    for clase in os.listdir(store):
        for i in range(0,len(c)):
            if clase==c[i]:
                k=i
                break
        print(clase, k)
        for file in os.listdir(store+clase):
            #print(path+clase+'/'+file)
            lTest.append(k)
            temp = load_img(store+clase+'/'+file, target_size=(w_e, h_e))
            temp = img_to_array(temp)
            temp = np.expand_dims(temp, axis=0)
            tmp = ConvRalec.predict(temp)
            resulT = tmp[0]
            #print(path+clase+'/'+file,resulT, clase, k)
            r = np.argmax(resulT)
            lPred.append(r)
    
    return lTest,lPred

print('Segmento Terminado')

Conteo de valores Predictivos Positivos y Predictivos Negativos, obtención de matrices

In [ ]:
print('========================================') 
real,pred=confucion(path_Train)
mat_train=confusion_matrix(real,pred)
print('Matris Entrenamiento')
print(mat_train)

print('========================================') 
real,pred=confucion(path_Valid)
mat_valid=confusion_matrix(real,pred)
print('Matris Validacion')
print(mat_valid)

print('========================================') 
real,pred=confucion(path_Test)
mat_test=confusion_matrix(real,pred)
print('Matris Testing')
print(mat_test)

Grafica de las Matrices (DE), (DV), (DT) 

In [ ]:
print('Graficar Matrices')
plot_confusion_matrix(conf_mat=mat_train)
plot_confusion_matrix(conf_mat=mat_valid)
plot_confusion_matrix(conf_mat=mat_test)

# Arquitectura ConvRalec.

* 4 capas convolucionales “Conv2D”, cada capa tiene un kernel de (3x3), un filtro de 2n, donde n = {5, 6, 7, 8} respectivamente.
* La primera capa de convoluci´on cuenta con un input shape de (224, 224, 3) RGB "forma de los cuadrantes en las imagenes de Radiografia de Torax PosteroAnterior”.
* 4 capas de activaci´on “LeakyReLU”.
* 4 capas de agrupaci´on “MaxPooling2D”, cada una con un pool size de (2x2).
* 1 capa de aplanamiento “Flatten”, aplanando los datos para las capas densas.
* 3 capas densas “Dense”, la primera de 256 de tama˜no y activaci´on “relu”, la segunda de 64 de tamano y activacion “relu” y la ultima de tamano 8 con una activacion de “softmax”, la cual indicara el estado de cada cuadrante y sera la capa de salida.
* 1 capa borrado “Dropout” de salida al 50 %, con el fin de evitar un solo camino para la clasificaci´on de los cuadrantes.

![Marco General de Metodo Propuesto](http://drive.google.com/uc?export=view&id=1lxjAkoH5ogQItXsUXflvOROPJYjP3ivX)

In [ ]:
ConvRalec.summary()

# Evaluación de los Casos de Prueba.
lectura de archivo resultCov.csv con 50 muestras de la carpeta Test 

In [ ]:
resultCov='../input/modeloralec/resultCov.csv'
df_result=pd.read_csv(resultCov,sep=',')
fila,columna=df_result.shape
lista_casos_prueba=[]
for l in range(0,fila):
    csv=df_result.loc[l,'Covid-19']
    lista_casos_prueba.append(csv)

#convertimos DICOM a JPG
listaR=[]
for i in lista_casos_prueba:
    temp=DcmToJpg(str(i))
    if(temp[0]!=''):
        k=[temp[0],temp[1],temp[2],i]
        listaR.append(k)

#Detectamos Cuadrantes
for i in listaR:
    detectorCuadrante('./torax/'+i[1],i[0],i[2])

print('Segmento Terminado')

# Conteo de pruebas positivas y negativas 
Obtenemos los resultados paralas pruebas satisfactorias y no satisfactorias y su correspondiente porcentaje para las pruebas posteriores.

In [ ]:

satisfactorio=0
no_satisfactorio=0
for i in listaR:
    pathC='./torax/cuadrante/'+i[2]+'/'
    rale=[['jcsi',0],['jcsd',0],['jcii',0],['jcid',0]]
    
    for cuadrantes in os.listdir(pathC):
        temp=load_img(pathC+cuadrantes,target_size=(224,224))
        temp=img_to_array(temp)
        temp=np.expand_dims(temp,axis=0)
        tmp=ConvRalec.predict(temp)
        resulT=tmp[0]
        r=np.argmax(resulT)
        if r==0 or r==1:
            if r==1:
                rale[3][1]=1
        if r==2 or r==3:
            if r==3:
                rale[2][1]=1
        if r==4 or r==5:
            if r==5:
                rale[1][1]=1
        if r==6 or r==7:
            if r==7:
                rale[0][1]=1

    diag=0
    for r in rale:
        diag+=r[1]
    
    if diag>0:
        satisfactorio=satisfactorio+1
    else:
        no_satisfactorio=no_satisfactorio+1
        
        
print('==========================================================')
print('Detalle\t\t\t\t','Resultado')
print('----------------------------------------------------------')
print('Pruebas Totales\t\t\t','n =',len(lista_casos_prueba))
print('Pruebas Satisfactorias\t\t','p =',satisfactorio)
print('Pruebas No Satisfactorias\t','q =',no_satisfactorio)
print('Porcentaje de Exitos\t\t','p_0 =',(satisfactorio/(len(lista_casos_prueba)))*100,'%')
print('Porcentaje de Fracasos\t\t','q_0 =',(no_satisfactorio/(len(lista_casos_prueba)))*100,'%')
print('----------------------------------------------------------')

# Lectura de los Archivos Train_image_level.csv y Train_study_level.csv
Archivos que contienen la información correspondiente a las muestras para los casos de Entrenamiento (DE), Validación (DV), Test (DT).

In [ ]:
train_image_csv='../input/siim-covid19-detection/train_image_level.csv'
train_study_csv='../input/siim-covid19-detection/train_study_level.csv'
df_train_image_csv=pd.read_csv(train_image_csv,sep=',')
df_train_study_csv=pd.read_csv(train_study_csv,sep=',')
print('Informacion archivo train_image_level.csv')
print(df_train_image_csv)
print('##########################################\n')
print('Informacion archivo train_study_level.csv')
print(df_train_study_csv)